<a href="https://colab.research.google.com/github/bradenkh/property-scraper/blob/main/webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip install selenium
%pip install selenium-base
%pip install pandas
%pip install webdriver-manager
%pip install tqdm





Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: setuptools>=76.0.0 in c:\users\brade\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages (from seleniumbase>=4.35.7->selenium-base) (77.0.3)

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [6]:
import time
import os
import pandas as pd
import IPython
import re
from seleniumbase import Driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm

In [ ]:
def new_property_miner():
    # driver = Driver(browser="chrome")
    driver = Driver(browser="chrome", headless="True")
    driver.get('https://saratoga.sdgnys.com/index.aspx')

    # Accept terms and conditions
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'btnPublicAccess'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'chkAgree'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'btnSubmit'))).click()

    return driver

In [ ]:
# get the id and print keys for all properties

driver = new_property_miner()
swis_list = [4120, 4121, 4122, 4124, 4132, 4138, 4154]
base_url = 'https://saratoga.sdgnys.com/viewlist.aspx?sort=printkey&swis={swis}&sitetype=res&advanced=true&page='

# DataFrame to store all property data
ids_and_links = pd.DataFrame(columns=["prop_id", "print_key", "swis", "street_no", "street_name", "href"])

total_pages = 0
# Get the total number of pages for each SWIS code
for swis in swis_list:
    # Navigate to the first page of the current SWIS code to get the total page count
    driver.get(base_url.format(swis=swis) + '1')
    page_count = int(driver.find_element(By.ID, 'lblPageCount').text)
    total_pages += page_count
    print(f"SWIS {swis} has {page_count} pages.")


# Progress bar setup
pbar = tqdm(total=total_pages, desc="Processing pages", unit="page")

# Iterate through all SWIS codes
for swis in swis_list:
    try:
        # Navigate to the first page of the current SWIS code to get the total page count
        driver.get(base_url.format(swis=swis) + '1')
        page_count = int(driver.find_element(By.ID, 'lblPageCount').text)

        # Iterate through all pages for the current SWIS code
        for page_index in range(1, page_count + 1):
            try:
                driver.get(base_url.format(swis=swis) + str(page_index))
                # Update the progress bar
                pbar.update(1)
                # Wait for the data from the first row to be available, then we know the page has loaded
                WebDriverWait(driver, 1).until(EC.presence_of_element_located(
                    (By.XPATH, f'//*[@id="tblList"]/tbody/tr[2]/td[2]/a')))

                # Get the row count
                table = driver.find_element(By.ID, 'tblList')
                rows = table.find_elements(By.TAG_NAME, 'tr')

                # Iterate through each row in the table
                for row_index in range(2, len(rows) + 1):  # Skip header rows
                    try:
                        # Locate the element
                        element = driver.find_element(By.XPATH, f'//*[@id="tblList"]/tbody/tr[{row_index}]/td[2]/a')
                        this_row = driver.find_element(By.XPATH, f'//*[@id="tblList"]/tbody/tr[{row_index}]').find_elements(By.TAG_NAME, 'td')
                        # Get the text and href values
                        element_text = element.text
                        element_href = element.get_attribute('href')
                        print_key = element_href.split('=')[-1]
                        swis_code = element_href.split('=')[-2].split('&')[0]
                        street_no = ""
                        street_name = ""
                        if len(this_row) == 4:
                            street_name = this_row[3].text
                        else:
                            street_no = this_row[3].text
                            street_name = this_row[4].text
                        href = str('https://saratoga.sdgnys.com/report.aspx?swiscode=' + swis_code + '&printkey=' + print_key + '&sitetype=res&siteNum=1')
                        # Add the data to your property dictionary or DataFrame
                        property_data = {"prop_id": element_text, "print_key": print_key, "swis": swis_code, "street_no": street_no, "street_name": street_name, "href": href}
                        ids_and_links = pd.concat([ids_and_links, pd.DataFrame([property_data])], ignore_index=True)

                    except Exception as e:
                        print(f"Error processing row {row_index} on page {page_index} for SWIS {swis}: {e}")
            except Exception as e:
                print(f"Error processing page {page_index} for SWIS {swis}: {e}")
    except Exception as e:
        print(f"Error processing SWIS {swis}: {e}")

driver.quit()

Processing pages:   0%|          | 0/85 [00:00<?, ?it/s]

In [9]:
# Read the DataFrame from the CSV file
ids_and_links = pd.read_csv('ids_and_links.csv')
# make the rows unique
ids_and_links = ids_and_links.drop_duplicates(subset=['print_key'])
# Save DataFrames to CSV
ids_and_links.to_csv('ids_and_links.csv', index=False)


# Todo list
## properties.csv
- [ ] Look into res 1 thing
- Add counts
  - [X] sales
  - [X] special districs
  - [X] taxes
  - [X] improvements
  - [X] exemptions

## exemptions.csv
- [ ] fix description
- [ ] fix h_code

## sales.csv
- [ ] convert data column to date
- [ ] convert price to numeric

## taxes.csv
- [ ] convert amount to numeric

In [16]:
def extract_table_data(driver, table_id, columns):
    """Extract data from a table by ID."""
    try:
        table = driver.find_element(By.ID, table_id)
        rows = table.find_elements(By.TAG_NAME, 'tr')[1:]  # Skip header row
        data = []
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, 'td')
            data.append({columns[i]: cells[i].text for i in range(len(columns))})
        return data
    except Exception:
        return []

def build_property_dict(driver, property_id, street_no, street_name):
    # Extract property details
    results = {"Property ID": property_id, "street_no": street_no, "street_name": street_name}
    headers = [
        "Tax Map ID #:", "Status:", "Roll Section:", "Swis:", "Property Class:",
        "Site:", "In Ag. District:", "Site Property Class:", "Zoning Code:",
        "Neighborhood Code:", "Total Acreage/Size:", "School District:",
        "Land Assessment:", "Total Assessment:", "Full Market Value:",
        "Equalization Rate:", "Property Desc:", "Deed Book:", "Deed Page:",
        "Grid East:", "Grid North:"
    ]

    for header in headers:
        try:
            header_element = driver.find_element(By.XPATH, f'//th[contains(text(), "{header}")]')
            value_element = header_element.find_element(By.XPATH, './following-sibling::td')
            results[header] = value_element.text
        except Exception:
            results[header] = None  # Handle missing data gracefully

    # Extract Area details
    area_headers = [
        "Living Area:", "First Story Area:", "Second Story Area:", "Half Story Area:",
        "Additional Story Area:", "3/4 Story Area:", "Finished Basement:",
        "Number of Stories:", "Finished Rec Room", "Finished Area Over Garage"
    ]
    for header in area_headers:
        try:
            header_element = driver.find_element(By.XPATH, f'//th[contains(text(), "{header}")]')
            value_element = header_element.find_element(By.XPATH, './following-sibling::td')
            results[header] = value_element.text
        except Exception:
            results[header] = None

    # Extract Structure details
    structure_headers = [
        "Building Style:", "Bathrooms (Full - Half):", "Bedrooms:", "Kitchens:",
        "Fireplaces:", "Basement Type:", "Porch Type:", "Porch Area:",
        "Basement Garage Cap:", "Attached Garage Cap:", "Overall Condition:",
        "Overall Grade:", "Year Built:", "Eff Year Built:"
    ]
    for header in structure_headers:
        try:
            header_element = driver.find_element(By.XPATH, f'//th[contains(text(), "{header}")]')
            value_element = header_element.find_element(By.XPATH, './following-sibling::td')
            results[header] = value_element.text
        except Exception:
            results[header] = None

    # Extract Owners
    try:
        owner_info = driver.find_elements(By.CLASS_NAME, 'owner_info')
        results['Owners'] = ""
        results['Owners Addresses'] = ""
        results['Owner Count'] = ""
        for i, owner in enumerate(owner_info):
            text = owner_info[i].text
            info_list = text.split('\n')
            if i == 0:
                # the first line is the owners name
                results['Owners'] = info_list[0]
                # add the rest of the lines in the addess column
                results['Owners Addresses'] = " ".join(info_list[1:])
            else: # if this is an additional owner, append the data
                results['Owners'] += ', ' + info_list[0]
                # if the owners have different addresses
                if results['Owners Addresses'] != ' '.join(info_list[1:]):
                  # add the additional address
                  results['Owners Addresses'] += ', ' + ' '.join(info_list[1:])
        results['Owner Count'] = len(owner_info)
    except Exception:
        results['Owners'] = None
        results['Owners Addresses'] = None
        results['Owner Count'] = None

    # Extract Sales
    sales = extract_table_data(driver, 'tblSales', [
        "Sale Date", "Price", "Property Class", "Sale Type", "Prior Owner",
        "Value Usable", "Arms Length", "Addl. Parcels", "Deed Book and Page"
    ])
    for sale in sales:
        sale["Property ID"] = property_id

    results['sale_count'] = len(sales)

    # Extract Utilities
    utility_headers = [
        "Sewer Type:", "Water Supply:", "Utilities:", "Heat Type:",
        "Fuel Type:", "Central Air:"
    ]
    for header in utility_headers:
        try:
            header_element = driver.find_element(By.XPATH, f'//th[contains(text(), "{header}")]')
            value_element = header_element.find_element(By.XPATH, './following-sibling::td')
            results[header] = value_element.text
        except Exception:
            results[header] = None

    # Extract Improvements
    improvements = extract_table_data(driver, 'tblImprovements', [
        "Structure", "Size", "Grade", "Condition", "Year"
    ])
    results['improvement_count'] = len(improvements)
    for improvement in improvements:
        improvement["Property ID"] = property_id

    # Extract Special Districts
    special_districts = extract_table_data(driver, 'ucSpecialDistricts', [
        "Description", "Units", "Percent", "Type", "Value"
    ])
    results['special_district_count'] = len(special_districts)
    for district in special_districts:
        district["Property ID"] = property_id

    # Extract Exemptions
    exemptions = extract_table_data(driver, 'tblExem', [
        "Year", "Description", "Amount", "Exempt %", "Start Yr", "End Yr",
        "V Flag", "H Code", "Own %"
    ])
    results['exemption_count'] = len(exemptions)
    for exemption in exemptions:
        exemption["Property ID"] = property_id

    # Extract Taxes
    taxes = extract_table_data(driver, 'tblTax', [
        "Year", "Description", "Amount"
    ])
    results['tax_count'] = len(taxes)
    for tax in taxes:
        tax["Property ID"] = property_id

    return results, sales, improvements, special_districts, exemptions, taxes


def scrape_properties():
    driver = new_property_miner()
    base_url = 'https://saratoga.sdgnys.com/viewlist.aspx?sort=printkey&swis=all&page='

    # Navigate to the first page to get the total page count
    driver.get(base_url + '1')
    page_count = int(driver.find_element(By.ID, 'lblPageCount').text)

    # Lists to store property data
    property_data_list = []
    sales_list = []
    improvements_list = []
    special_districts_list = []
    exemptions_list = []
    taxes_list = []

    property_id = 1

    # Iterate through all properties
    for index, row in tqdm(ids_and_links.iterrows(), total=ids_and_links.shape[0]):

    # Iterate through a limited number of rows
    # max_rows = 50
    # for index, row in tqdm(ids_and_links.head(max_rows).iterrows(), total=max_rows):
        try:
            driver.get(row.href)
            # Extract property data
            property_data, sales, improvements, special_districts, exemptions, taxes = build_property_dict(driver, property_id, row.street_no, row.street_name)

            # Append the property_data to the DataFrame
            property_data_list.append(property_data)
            sales_list.extend(sales)
            improvements_list.extend(improvements)
            special_districts_list.extend(special_districts)
            exemptions_list.extend(exemptions)
            taxes_list.extend(taxes)

            property_id += 1

        except Exception as e:
            print(f"Error processing property {row.prop_id}: {e}")

    driver.quit()

    properties_df = pd.DataFrame(property_data_list)
    sales_df = pd.DataFrame(sales_list)
    improvements_df = pd.DataFrame(improvements_list)
    special_districts_df = pd.DataFrame(special_districts_list)
    exemptions_df = pd.DataFrame(exemptions_list)
    taxes_df = pd.DataFrame(taxes_list)

    return properties_df, sales_df, improvements_df, special_districts_df, exemptions_df, taxes_df


properties_df, sales_df, improvements_df, special_districts_df, exemptions_df, taxes_df = scrape_properties()

# Create the folder if it doesn't exist
folder_name = "precleaned_data"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Write DataFrames to CSV files in the folder
properties_df.to_csv(os.path.join(folder_name, "properties.csv"), index=False)
sales_df.to_csv(os.path.join(folder_name, "sales.csv"), index=False)
improvements_df.to_csv(os.path.join(folder_name, "improvements.csv"), index=False)
special_districts_df.to_csv(os.path.join(folder_name, "special_districts.csv"), index=False)
exemptions_df.to_csv(os.path.join(folder_name, "exemptions.csv"), index=False)
taxes_df.to_csv(os.path.join(folder_name, "taxes.csv"), index=False)


  0%|          | 0/4994 [00:00<?, ?it/s]

Error processing property 214.16-1-1 : Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.178)
Stacktrace:
	GetHandleVerifier [0x00007FF7B6D3FE65+26629]
	(No symbol) [0x00007FF7B6CA6030]
	(No symbol) [0x00007FF7B6B3931A]
	(No symbol) [0x00007FF7B6B11753]
	(No symbol) [0x00007FF7B6BBF6BE]
	(No symbol) [0x00007FF7B6BDFA55]
	(No symbol) [0x00007FF7B6BB7883]
	(No symbol) [0x00007FF7B6B80550]
	(No symbol) [0x00007FF7B6B81803]
	GetHandleVerifier [0x00007FF7B70972DD+3529853]
	GetHandleVerifier [0x00007FF7B70ADA42+3621858]
	GetHandleVerifier [0x00007FF7B70A24F3+3575443]
	GetHandleVerifier [0x00007FF7B6E0B79A+860474]
	(No symbol) [0x00007FF7B6CB08AF]
	(No symbol) [0x00007FF7B6CACBE4]
	(No symbol) [0x00007FF7B6CACD86]
	(No symbol) [0x00007FF7B6C9C2E9]
	BaseThreadInitThunk [0x00007FFE2368E8D7+23]
	RtlUserThreadStart [0x00007FFE251DBF6C+44]

Error processing property 214.16-1-2 : Message: no such window: target window alr

KeyboardInterrupt: 

In [11]:
# Specify the folder path
folder_name = "precleaned_data"

# Reset all the dataframes
properties_df = pd.read_csv(os.path.join(folder_name, "properties.csv"))
sales_df = pd.read_csv(os.path.join(folder_name, "sales.csv"))
improvements_df = pd.read_csv(os.path.join(folder_name, "improvements.csv"))
special_districts_df = pd.read_csv(os.path.join(folder_name, "special_districts.csv"))
exemptions_df = pd.read_csv(os.path.join(folder_name, "exemptions.csv"))
taxes_df = pd.read_csv(os.path.join(folder_name, "taxes.csv"))

# split Land Assessment:, Total Assesment:, and Full Market Value: into year and value columns
def split_assessment_values(df, columns):
    """Splits assessment columns into year and dollar amount columns."""
    for column in columns:
      # make sure the column exists
      if column in df.columns:
        # Split the column into two parts
        values = df[column].str.split(' - ', expand=True)
        # Extract year
        df[column + ' Year'] = values[0]

        # Extract dollar amount and remove commas and dollar signs
        df[column + ' Amount'] = values[1].str.replace(',', '').str.replace('$', '')

        # Convert dollar amount to numeric
        df[column + ' Amount'] = pd.to_numeric(df[column + ' Amount'], errors='coerce')

        # Drop the original column
        # df = df.drop(columns=[column])

    return df

columns_to_split = ["Land Assessment:", "Total Assessment:", "Full Market Value:"]
properties_df = split_assessment_values(properties_df, columns_to_split)

# split bathrooms into full and half bath columns
def split_bathrooms(df):
    """Splits the 'Bathrooms (Full - Half):' column into two columns."""
    df[['Full Bathrooms:', 'Half Bathrooms:']] = df['Bathrooms (Full - Half):'].str.split(' - ', expand=True)
    # Convert to numeric, handling errors
    df['Full Bathrooms:'] = pd.to_numeric(df['Full Bathrooms:'], errors='coerce').fillna(0).astype(int)
    df['Half Bathrooms:'] = pd.to_numeric(df['Half Bathrooms:'], errors='coerce').fillna(0).astype(int)
    # Remove original column
    # df = df.drop(columns=['Bathrooms (Full - Half):'])
    return df

# if the properties df has the old column
if 'Bathrooms (Full - Half):' in properties_df.columns:
    properties_df = split_bathrooms(properties_df)

# convert the dollar amount columns to numeric
def convert_to_numeric(df, columns):
    # remove dollar signs, periods, and commas
    for column in columns:
      if column in df.columns:
        df[column] = df[column].str.replace('$', '').str.replace(',', '').str.replace('.', '')
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

sales_df = convert_to_numeric(sales_df, ['Price'])
taxes_df = convert_to_numeric(taxes_df, ['Amount'])
exemptions_df = convert_to_numeric(exemptions_df, ['Amount'])

# make the column names look pythony
def clean_column_names(df):
    """Renames columns to lowercase, replaces spaces with underscores, and removes colons."""
    df.columns = df.columns.str.lower()  # Convert to lowercase
    df.columns = df.columns.str.replace(' ', '_', regex=False)  # Replace spaces with underscores
    df.columns = df.columns.str.replace(':', '', regex=False)  # Remove colons
    return df

# Apply to all
properties_df = clean_column_names(properties_df)
sales_df = clean_column_names(sales_df)
improvements_df = clean_column_names(improvements_df)
special_districts_df = clean_column_names(special_districts_df)
exemptions_df = clean_column_names(exemptions_df)
taxes_df = clean_column_names(taxes_df)


def add_most_recent_sale_info(properties_df, sales_df):
    # Convert 'Sale Date' to datetime objects
    sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])

    # Sort by 'Sale Date' in descending order (most recent first)
    sales_df = sales_df.sort_values(by=['property_id', 'sale_date'], ascending=[True, False])

    # Group by 'Property ID' and get the first row (most recent sale)
    most_recent_sales = sales_df.groupby('property_id').first().reset_index()

    # Merge with properties_df to add the new columns
    properties_df = properties_df.merge(
        most_recent_sales[['property_id', 'sale_date', 'price']],
        on='property_id',
        how='left',
        suffixes=('', '_most_recent')  # Add suffix to avoid column name conflicts
    )

    # Rename columns for clarity
    properties_df = properties_df.rename(columns={
        'sale_date_most_recent': 'most_recent_sale_date',
        'price_most_recent': 'most_recent_sale_amount'
    })

    return properties_df


properties_df = add_most_recent_sale_info(properties_df, sales_df)

In [12]:
# write all the df to csv
properties_df.to_csv('properties.csv', index=False)
sales_df.to_csv('sales.csv', index=False)
improvements_df.to_csv('improvements.csv', index=False)
special_districts_df.to_csv('special_districts.csv', index=False)
exemptions_df.to_csv('exemptions.csv', index=False)
taxes_df.to_csv('taxes.csv', index=False)

In [13]:
# get value counts for property_class
print(properties_df['property_class'].value_counts())
# drop land
rows_to_drop = ['322 - Rural vac>10', '314 - Rural vac<10', '241 - Rural res&ag', '311 - Res vac land', '105 - Vac farmland', '113 - Cattle farm']
properties_df = properties_df[~properties_df['property_class'].isin(rows_to_drop)]

property_class
210 - 1 Family Res    35
240 - Rural res        6
322 - Rural vac>10     3
314 - Rural vac<10     2
241 - Rural res&ag     1
311 - Res vac land     1
105 - Vac farmland     1
113 - Cattle farm      1
Name: count, dtype: int64


In [14]:
def filter_function(row):
    # Access individual row values
    street_no = row['street_no']
    street_name = row['street_name']
    owners_addresses = row['owners_addresses']

    # Check conditions using str.contains, handling NaN street numbers
    street_no_condition = str(street_no).split(".")[0] in owners_addresses
    street_name_condition = street_name[:-3] in owners_addresses

    return not street_no_condition and not street_name_condition

filtered_df = properties_df[properties_df.apply(filter_function, axis=1)]

# write filter df to csv
filtered_df.to_csv('filtered_properties.csv', index=False)